# Startup Name Generator

### You are launching a new start-up company. Using RNN LSTM let's generate a cool start-up name.

Go to [Training](#training)

Go to [Generating Names](#generating)

Go to [Results](#result)

In [1]:
# Packages required.
import tensorflow as tf
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences

import numpy as np
import pandas as pd
import random
import pprint
import requests
from bs4 import BeautifulSoup

Using TensorFlow backend.


### Let's get start-up names established in San Francisco.

I got a list of names from https://www.startups-list.com/. The site has lists of start-up names in other cities as well.

Let's extract the data out of the HTML and parse the HTML document.

In [2]:
headers = {'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"}
url = 'https://sanfrancisco.startups-list.com/'
page = requests.get(url, headers = headers)
page

<Response [200]>

In [3]:
soup = BeautifulSoup(page.text, 'lxml') #using lxml parser. You can use Python’s html.parser.

In [4]:
SFnames = []
for name in soup.findAll(property='name'):
    SFnames.append(name.text.lower().replace('\n',''))

In [5]:
len(SFnames)

2329

In [6]:
max_len = 0
for name in SFnames:
    max_len = max(max_len, len(name))

In [7]:
max_len # = Tx the size of the sequence input. I'll pad them if they are shorter than this length.

39

In [8]:
SFnames_str = ''
for name in soup.findAll(property='name'):
    SFnames_str = SFnames_str+name.text.replace('\n','')+'\n'

In [9]:
data= SFnames_str.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 23255 total characters and 55 unique characters in your data.


In [10]:
chars = sorted(chars)
print(chars)

['\n', ' ', '!', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '@', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '✈', '️']


- There are 26 alphabets, 10 numbers, "\n" (newline character) as well as some special characters.
- There are some uncommon special characters such as '\xa0' or '✈' but a neural network is robust to these noises so I leave it as it is.
- "\n" plays a role "End of name" token.

Let's make a hash tag to convert a character to an index and vice versa.
- `char_to_ix`: a python dictionary to map each character to an index.
- `ix_to_char`: a second python dictionary that maps each index back to the corresponding character. 

In [11]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(ix_to_char)

{   0: '\n',
    1: ' ',
    2: '!',
    3: '&',
    4: "'",
    5: '(',
    6: ')',
    7: '+',
    8: ',',
    9: '-',
    10: '.',
    11: '/',
    12: '0',
    13: '1',
    14: '2',
    15: '3',
    16: '4',
    17: '5',
    18: '6',
    19: '7',
    20: '8',
    21: '9',
    22: ':',
    23: '@',
    24: '[',
    25: ']',
    26: 'a',
    27: 'b',
    28: 'c',
    29: 'd',
    30: 'e',
    31: 'f',
    32: 'g',
    33: 'h',
    34: 'i',
    35: 'j',
    36: 'k',
    37: 'l',
    38: 'm',
    39: 'n',
    40: 'o',
    41: 'p',
    42: 'q',
    43: 'r',
    44: 's',
    45: 't',
    46: 'u',
    47: 'v',
    48: 'w',
    49: 'x',
    50: 'y',
    51: 'z',
    52: '\xa0',
    53: '✈',
    54: '️'}


In [12]:
n_values = len(char_to_ix)

I'm going to train a model that predicts the next character in a way that is similar to the start-up names that it's trained on.

## Building the model

- The model takes input X of shape $(m, T_x, n_{values})$ and labels Y of shape $(T_y, m, n_{values})$. 
- We will use an LSTM with hidden states that have $n_{a} = 64$ dimensions.

We are generating a sequence of characters so we generate them one at a time using $x^{\langle t\rangle} = y^{\langle t-1 \rangle}$.
- The input at time "t" is the prediction at the previous time step "t-1".

Let's define the layers objects we need as global variables.
- weights and bias are globally defined and shareable

In [13]:
# number of dimensions for the hidden state of each LSTM cell.
n_a = 64 
n_values = len(char_to_ix) # number of music values
reshapor = Reshape((1, n_values))  # to help reshape tensors  
LSTM_cell = LSTM(n_a, return_state = True)   # build a LSTM cell at time t
densor = Dense(n_values, activation='softmax')  # Dense layer after each LSTM cell

In [14]:
def name_model(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the training data 
    
    Returns:
    model -- a keras instance model with n_a activations
    """
    
    X = Input(shape=(Tx, n_values)) # Input Layer

    a0 = Input(shape=(n_a,), name='a0') # Initial hidden state
    c0 = Input(shape=(n_a,), name='c0') # Initial cell state
    a = a0
    c = c0

    outputs = []
    
    for t in range(Tx):
        
        x = Lambda(lambda x:x[:,t,:], output_shape=(n_values,))(X)
        x = reshapor(x)
        a, _, c = LSTM_cell(inputs=x, initial_state=[a, c])
        out = densor(a)
        outputs.append(out)

    model = Model(inputs=[X, a0, c0], outputs=outputs)

    return model

In [28]:
model = name_model(Tx = max_len , n_a = n_a, n_values = n_values)

In [16]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 39, 55)       0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 55)           0           input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 55)        0           lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   
                                                                 lambda_3[0][0]                   
                                                                 lambda_4[0][0]             

Let's compile the model using the Adam optimizer and the categorical cross-entropy loss function.

In [29]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

<a id='training' />

## Training

Let's initialize `a0` and `c0` for the LSTM's initial state to be zero. 

In [18]:
m = len(SFnames)
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

We'll make our input X and label Y such that

- `X` is a (m, $T_x$, $n_{value}$) dimensional array.
    - We have m training examples, each of which is a snippet of $T_x$ musical values. 
    - At each time step, the input is one of 78 different possible values, represented as a one-hot vector. 
        - For example, X[i,t,:] is a one-hot vector representing the value of the i-th example at time t. 
- `Y` is a $(T_y, m, n_{value})$ dimensional array (reordered to be more convenient to feed into the LSTM).
- `Y` is basically an one-time-step pushed array of X and reordered.
    - we're using the previous values to predict the next value, so our sequence model will try to predict $y^{\langle t \rangle}$ given $x^{\langle 1\rangle}, \ldots, x^{\langle t \rangle}$. 

- We will turn `Y` into a list, since the cost function expects `Y` to be provided in this format. Each of the list items is of shape (m, n_value). 

In [19]:
X = []
for name in SFnames:
    X.append([-1]+[char_to_ix[c] for c in name])

# pad after the sequence if they are shorter than max_len
X = pad_sequences(X, maxlen=max_len, padding='post',  value=-1)

In [20]:
X

array([[-1, 34, 39, ..., -1, -1, -1],
       [-1, 44, 26, ..., -1, -1, -1],
       [-1, 44, 42, ..., -1, -1, -1],
       ...,
       [-1, 36, 30, ..., -1, -1, -1],
       [-1, 41, 43, ..., -1, -1, -1],
       [-1, 27, 46, ..., -1, -1, -1]], dtype=int32)

In [21]:
X = to_categorical(X)

In [22]:
X.shape # check the shape

(2329, 39, 55)

In [23]:
Y = []
for name in SFnames:
    Y.append([char_to_ix[c] for c in name]+[char_to_ix['\n']])

# pad Y in the same way
Y = pad_sequences(Y, maxlen=max_len, padding='post',  value=-1)
Y = to_categorical(Y)
Y = np.transpose(Y,[1,0,2])

In [25]:
Y.shape # check the shape

(39, 2329, 55)

In [26]:
max_len

39

Let's fit the model. I'll train for 200 epochs.

In [30]:
model.fit([X, a0, c0], list(Y), epochs=200, verbose = 1)

Epoch 1/200
2329/2329 [==============================] - 17s 7ms/step - loss: 26.8422 - dense_1_loss: 7.7798e-04 - dense_1_accuracy: 0.1224 - dense_1_accuracy_1: 0.2035 - dense_1_accuracy_2: 0.1473 - dense_1_accuracy_3: 0.1619 - dense_1_accuracy_4: 0.1829 - dense_1_accuracy_5: 0.2293 - dense_1_accuracy_6: 0.3362 - dense_1_accuracy_7: 0.4616 - dense_1_accuracy_8: 0.6011 - dense_1_accuracy_9: 0.7042 - dense_1_accuracy_10: 0.7900 - dense_1_accuracy_11: 0.8394 - dense_1_accuracy_12: 0.8836 - dense_1_accuracy_13: 0.9021 - dense_1_accuracy_14: 0.9223 - dense_1_accuracy_15: 0.9377 - dense_1_accuracy_16: 0.9536 - dense_1_accuracy_17: 0.9588 - dense_1_accuracy_18: 0.9682 - dense_1_accuracy_19: 0.9717 - dense_1_accuracy_20: 0.9781 - dense_1_accuracy_21: 0.9837 - dense_1_accuracy_22: 0.9871 - dense_1_accuracy_23: 0.9871 - dense_1_accuracy_24: 0.9888 - dense_1_accuracy_25: 0.9893 - dense_1_accuracy_26: 0.9897 - dense_1_accuracy_27: 0.9918 - dense_1_accuracy_28: 0.9940 - dense_1_accuracy_29: 0.9953

2329/2329 [==============================] - 2s 1ms/step - loss: 24.6575 - dense_1_loss: 5.0080e-04 - dense_1_accuracy: 0.1224 - dense_1_accuracy_1: 0.2563 - dense_1_accuracy_2: 0.2095 - dense_1_accuracy_3: 0.2134 - dense_1_accuracy_4: 0.1971 - dense_1_accuracy_5: 0.2688 - dense_1_accuracy_6: 0.3697 - dense_1_accuracy_7: 0.4985 - dense_1_accuracy_8: 0.6243 - dense_1_accuracy_9: 0.7256 - dense_1_accuracy_10: 0.8154 - dense_1_accuracy_11: 0.8553 - dense_1_accuracy_12: 0.9000 - dense_1_accuracy_13: 0.9154 - dense_1_accuracy_14: 0.9339 - dense_1_accuracy_15: 0.9472 - dense_1_accuracy_16: 0.9588 - dense_1_accuracy_17: 0.9635 - dense_1_accuracy_18: 0.9747 - dense_1_accuracy_19: 0.9742 - dense_1_accuracy_20: 0.9790 - dense_1_accuracy_21: 0.9863 - dense_1_accuracy_22: 0.9901 - dense_1_accuracy_23: 0.9897 - dense_1_accuracy_24: 0.9923 - dense_1_accuracy_25: 0.9910 - dense_1_accuracy_26: 0.9923 - dense_1_accuracy_27: 0.9923 - dense_1_accuracy_28: 0.9944 - dense_1_accuracy_29: 0.9961 - dense_1_ac

2329/2329 [==============================] - 2s 1ms/step - loss: 23.9776 - dense_1_loss: 5.0306e-04 - dense_1_accuracy: 0.1224 - dense_1_accuracy_1: 0.2636 - dense_1_accuracy_2: 0.2276 - dense_1_accuracy_3: 0.2396 - dense_1_accuracy_4: 0.2280 - dense_1_accuracy_5: 0.2821 - dense_1_accuracy_6: 0.3839 - dense_1_accuracy_7: 0.5079 - dense_1_accuracy_8: 0.6337 - dense_1_accuracy_9: 0.7385 - dense_1_accuracy_10: 0.8240 - dense_1_accuracy_11: 0.8609 - dense_1_accuracy_12: 0.9030 - dense_1_accuracy_13: 0.9206 - dense_1_accuracy_14: 0.9420 - dense_1_accuracy_15: 0.9476 - dense_1_accuracy_16: 0.9635 - dense_1_accuracy_17: 0.9682 - dense_1_accuracy_18: 0.9755 - dense_1_accuracy_19: 0.9755 - dense_1_accuracy_20: 0.9802 - dense_1_accuracy_21: 0.9875 - dense_1_accuracy_22: 0.9893 - dense_1_accuracy_23: 0.9910 - dense_1_accuracy_24: 0.9931 - dense_1_accuracy_25: 0.9914 - dense_1_accuracy_26: 0.9931 - dense_1_accuracy_27: 0.9923 - dense_1_accuracy_28: 0.9944 - dense_1_accuracy_29: 0.9961 - dense_1_ac

2329/2329 [==============================] - 2s 1ms/step - loss: 23.5493 - dense_1_loss: 5.3844e-04 - dense_1_accuracy: 0.1224 - dense_1_accuracy_1: 0.2684 - dense_1_accuracy_2: 0.2456 - dense_1_accuracy_3: 0.2482 - dense_1_accuracy_4: 0.2417 - dense_1_accuracy_5: 0.2958 - dense_1_accuracy_6: 0.3912 - dense_1_accuracy_7: 0.5118 - dense_1_accuracy_8: 0.6462 - dense_1_accuracy_9: 0.7471 - dense_1_accuracy_10: 0.8252 - dense_1_accuracy_11: 0.8690 - dense_1_accuracy_12: 0.9064 - dense_1_accuracy_13: 0.9227 - dense_1_accuracy_14: 0.9476 - dense_1_accuracy_15: 0.9506 - dense_1_accuracy_16: 0.9657 - dense_1_accuracy_17: 0.9725 - dense_1_accuracy_18: 0.9785 - dense_1_accuracy_19: 0.9772 - dense_1_accuracy_20: 0.9828 - dense_1_accuracy_21: 0.9888 - dense_1_accuracy_22: 0.9897 - dense_1_accuracy_23: 0.9910 - dense_1_accuracy_24: 0.9931 - dense_1_accuracy_25: 0.9918 - dense_1_accuracy_26: 0.9936 - dense_1_accuracy_27: 0.9931 - dense_1_accuracy_28: 0.9944 - dense_1_accuracy_29: 0.9966 - dense_1_ac

2329/2329 [==============================] - 2s 1ms/step - loss: 23.2581 - dense_1_loss: 6.0577e-04 - dense_1_accuracy: 0.1224 - dense_1_accuracy_1: 0.2684 - dense_1_accuracy_2: 0.2555 - dense_1_accuracy_3: 0.2662 - dense_1_accuracy_4: 0.2512 - dense_1_accuracy_5: 0.3023 - dense_1_accuracy_6: 0.4032 - dense_1_accuracy_7: 0.5191 - dense_1_accuracy_8: 0.6539 - dense_1_accuracy_9: 0.7523 - dense_1_accuracy_10: 0.8295 - dense_1_accuracy_11: 0.8720 - dense_1_accuracy_12: 0.9116 - dense_1_accuracy_13: 0.9244 - dense_1_accuracy_14: 0.9476 - dense_1_accuracy_15: 0.9536 - dense_1_accuracy_16: 0.9682 - dense_1_accuracy_17: 0.9755 - dense_1_accuracy_18: 0.9785 - dense_1_accuracy_19: 0.9785 - dense_1_accuracy_20: 0.9841 - dense_1_accuracy_21: 0.9893 - dense_1_accuracy_22: 0.9888 - dense_1_accuracy_23: 0.9910 - dense_1_accuracy_24: 0.9931 - dense_1_accuracy_25: 0.9918 - dense_1_accuracy_26: 0.9940 - dense_1_accuracy_27: 0.9931 - dense_1_accuracy_28: 0.9944 - dense_1_accuracy_29: 0.9966 - dense_1_ac

...

2329/2329 [==============================] - 2s 1ms/step - loss: 21.3590 - dense_1_loss: 1.3100e-04 - dense_1_accuracy: 0.1224 - dense_1_accuracy_1: 0.2679 - dense_1_accuracy_2: 0.2976 - dense_1_accuracy_3: 0.3276 - dense_1_accuracy_4: 0.3130 - dense_1_accuracy_5: 0.3474 - dense_1_accuracy_6: 0.4611 - dense_1_accuracy_7: 0.5754 - dense_1_accuracy_8: 0.6973 - dense_1_accuracy_9: 0.7888 - dense_1_accuracy_10: 0.8562 - dense_1_accuracy_11: 0.8961 - dense_1_accuracy_12: 0.9261 - dense_1_accuracy_13: 0.9438 - dense_1_accuracy_14: 0.9571 - dense_1_accuracy_15: 0.9631 - dense_1_accuracy_16: 0.9751 - dense_1_accuracy_17: 0.9824 - dense_1_accuracy_18: 0.9863 - dense_1_accuracy_19: 0.9837 - dense_1_accuracy_20: 0.9893 - dense_1_accuracy_21: 0.9918 - dense_1_accuracy_22: 0.9918 - dense_1_accuracy_23: 0.9927 - dense_1_accuracy_24: 0.9953 - dense_1_accuracy_25: 0.9940 - dense_1_accuracy_26: 0.9953 - dense_1_accuracy_27: 0.9966 - dense_1_accuracy_28: 0.9953 - dense_1_accuracy_29: 0.9983 - dense_1_ac

2329/2329 [==============================] - 2s 1ms/step - loss: 21.3300 - dense_1_loss: 1.2679e-04 - dense_1_accuracy: 0.1224 - dense_1_accuracy_1: 0.2701 - dense_1_accuracy_2: 0.2971 - dense_1_accuracy_3: 0.3332 - dense_1_accuracy_4: 0.3143 - dense_1_accuracy_5: 0.3478 - dense_1_accuracy_6: 0.4650 - dense_1_accuracy_7: 0.5788 - dense_1_accuracy_8: 0.6943 - dense_1_accuracy_9: 0.7870 - dense_1_accuracy_10: 0.8583 - dense_1_accuracy_11: 0.8961 - dense_1_accuracy_12: 0.9270 - dense_1_accuracy_13: 0.9438 - dense_1_accuracy_14: 0.9566 - dense_1_accuracy_15: 0.9631 - dense_1_accuracy_16: 0.9747 - dense_1_accuracy_17: 0.9820 - dense_1_accuracy_18: 0.9863 - dense_1_accuracy_19: 0.9833 - dense_1_accuracy_20: 0.9893 - dense_1_accuracy_21: 0.9914 - dense_1_accuracy_22: 0.9927 - dense_1_accuracy_23: 0.9918 - dense_1_accuracy_24: 0.9953 - dense_1_accuracy_25: 0.9940 - dense_1_accuracy_26: 0.9948 - dense_1_accuracy_27: 0.9966 - dense_1_accuracy_28: 0.9953 - dense_1_accuracy_29: 0.9983 - dense_1_ac

2329/2329 [==============================] - 2s 1ms/step - loss: 21.3029 - dense_1_loss: 1.3785e-04 - dense_1_accuracy: 0.1224 - dense_1_accuracy_1: 0.2701 - dense_1_accuracy_2: 0.2984 - dense_1_accuracy_3: 0.3336 - dense_1_accuracy_4: 0.3143 - dense_1_accuracy_5: 0.3499 - dense_1_accuracy_6: 0.4667 - dense_1_accuracy_7: 0.5788 - dense_1_accuracy_8: 0.6969 - dense_1_accuracy_9: 0.7888 - dense_1_accuracy_10: 0.8579 - dense_1_accuracy_11: 0.8957 - dense_1_accuracy_12: 0.9266 - dense_1_accuracy_13: 0.9442 - dense_1_accuracy_14: 0.9566 - dense_1_accuracy_15: 0.9626 - dense_1_accuracy_16: 0.9751 - dense_1_accuracy_17: 0.9824 - dense_1_accuracy_18: 0.9858 - dense_1_accuracy_19: 0.9837 - dense_1_accuracy_20: 0.9906 - dense_1_accuracy_21: 0.9914 - dense_1_accuracy_22: 0.9923 - dense_1_accuracy_23: 0.9918 - dense_1_accuracy_24: 0.9953 - dense_1_accuracy_25: 0.9940 - dense_1_accuracy_26: 0.9948 - dense_1_accuracy_27: 0.9966 - dense_1_accuracy_28: 0.9953 - dense_1_accuracy_29: 0.9983 - dense_1_ac

2329/2329 [==============================] - 2s 1ms/step - loss: 21.2745 - dense_1_loss: 1.2128e-04 - dense_1_accuracy: 0.1224 - dense_1_accuracy_1: 0.2701 - dense_1_accuracy_2: 0.2988 - dense_1_accuracy_3: 0.3349 - dense_1_accuracy_4: 0.3139 - dense_1_accuracy_5: 0.3491 - dense_1_accuracy_6: 0.4663 - dense_1_accuracy_7: 0.5814 - dense_1_accuracy_8: 0.6973 - dense_1_accuracy_9: 0.7900 - dense_1_accuracy_10: 0.8583 - dense_1_accuracy_11: 0.8961 - dense_1_accuracy_12: 0.9266 - dense_1_accuracy_13: 0.9450 - dense_1_accuracy_14: 0.9566 - dense_1_accuracy_15: 0.9635 - dense_1_accuracy_16: 0.9751 - dense_1_accuracy_17: 0.9828 - dense_1_accuracy_18: 0.9863 - dense_1_accuracy_19: 0.9837 - dense_1_accuracy_20: 0.9893 - dense_1_accuracy_21: 0.9914 - dense_1_accuracy_22: 0.9927 - dense_1_accuracy_23: 0.9918 - dense_1_accuracy_24: 0.9953 - dense_1_accuracy_25: 0.9940 - dense_1_accuracy_26: 0.9953 - dense_1_accuracy_27: 0.9966 - dense_1_accuracy_28: 0.9953 - dense_1_accuracy_29: 0.9983 - dense_1_ac

2329/2329 [==============================] - 2s 1ms/step - loss: 21.2491 - dense_1_loss: 1.0941e-04 - dense_1_accuracy: 0.1224 - dense_1_accuracy_1: 0.2701 - dense_1_accuracy_2: 0.3014 - dense_1_accuracy_3: 0.3362 - dense_1_accuracy_4: 0.3164 - dense_1_accuracy_5: 0.3478 - dense_1_accuracy_6: 0.4697 - dense_1_accuracy_7: 0.5805 - dense_1_accuracy_8: 0.6960 - dense_1_accuracy_9: 0.7900 - dense_1_accuracy_10: 0.8587 - dense_1_accuracy_11: 0.8970 - dense_1_accuracy_12: 0.9261 - dense_1_accuracy_13: 0.9450 - dense_1_accuracy_14: 0.9571 - dense_1_accuracy_15: 0.9639 - dense_1_accuracy_16: 0.9751 - dense_1_accuracy_17: 0.9828 - dense_1_accuracy_18: 0.9858 - dense_1_accuracy_19: 0.9837 - dense_1_accuracy_20: 0.9897 - dense_1_accuracy_21: 0.9923 - dense_1_accuracy_22: 0.9927 - dense_1_accuracy_23: 0.9918 - dense_1_accuracy_24: 0.9953 - dense_1_accuracy_25: 0.9940 - dense_1_accuracy_26: 0.9953 - dense_1_accuracy_27: 0.9966 - dense_1_accuracy_28: 0.9953 - dense_1_accuracy_29: 0.9983 - dense_1_ac

<a id='generating' />

## Generating names

Let's build a inference model to sample the names from trained model.

In [46]:
def one_hot(x, temperature = 0.01):
    x = tf.random.categorical(x/temperature, 1)
    # temperature adjust how diverse the output could be.
    # the higher temperature, the more surprising result
    x = tf.one_hot(indices=x, depth=n_values) 
    return x

# Use below if want argmax for the sampling.
# It will generate the same result every time for the same initial states.

# def one_hot(x):
#     x = K.argmax(x)
#     x = tf.one_hot(indices=x, depth=n_values) 
#     x = RepeatVector(1)(x)
#     return x

def inference_model(LSTM_cell, densor, n_values = n_values, n_a = n_a, Ty = max_len):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, number of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    model -- Keras model instance
    """
    x0 = Input(shape=(1, n_values))
    
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    outputs = []

    for t in range(Ty):
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        out = densor(a)
        outputs.append(out)
        x = Lambda(one_hot)(out)
        
    model = Model(inputs=[x0, a0, c0], outputs=outputs)
       
    return model

In [47]:
output_model = inference_model(LSTM_cell, densor, n_values = n_values, n_a = n_a, Ty = max_len)

In [33]:
# Check the inference model
output_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1, 55)        0                                            
__________________________________________________________________________________________________
a0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
c0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 64), (None,  30720       input_3[0][0]                    
                                                                 a0[0][0]                   

                                                                 lambda_105[0][0]                 
                                                                 lstm_1[104][0]                   
                                                                 lstm_1[104][2]                   
                                                                 lambda_106[0][0]                 
                                                                 lstm_1[105][0]                   
                                                                 lstm_1[105][2]                   
                                                                 lambda_107[0][0]                 
                                                                 lstm_1[106][0]                   
                                                                 lstm_1[106][2]                   
                                                                 lambda_108[0][0]                 
          

- Initialize states

In [43]:
x_initializer = np.zeros((1, 1, n_values))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

In [44]:
def predict_and_sample(output_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    output_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, n_value), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, n_value), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    pred = output_model.predict([x_initializer, a_initializer, c_initializer])
    indices = np.argmax(pred, axis =-1)
    results = to_categorical(indices)
    
    return results, indices 

<a id='result' />

## Results

Let's get the results.

### - Argmax

In [45]:
# for this cell I used the one_hot function with K.argmax
results, indices = predict_and_sample(output_model, x_initializer, a_initializer, c_initializer)
for i in indices.flatten():
    if ix_to_char[i] == '\n':
        break
    print(ix_to_char[i],end="")

apprest

<font color = 'darkblue'>'Apprest' is the most optimized name of the model. I think it is a good name. It could be a name for a new IT company, for a new game, or for a biotech company.</font>

### - Random sampling with zero initialization

Using the one_hot function with random sampling, let's generate some awesome sample names.

In [48]:
iteration = 20
for _ in range(iteration):
    x_initializer = np.zeros((1, 1, n_values))
    a_initializer = np.zeros((1, n_a))
    c_initializer = np.zeros((1, n_a))
    results, indices = predict_and_sample(output_model, x_initializer, a_initializer, c_initializer)
    for i in indices.flatten():
        if ix_to_char[i] == '\n':
            print()
            break
        print(ix_to_char[i],end="")
#     print("np.argmax(results[12]) =", np.argmax(results[12]))
#     print("np.argmax(results[17]) =", np.argmax(results[17]))
#     print("list(indices[12:18]) =", list(indices[12:18]))

anppreese
apprest
apprrest
apprest
aondersound
aondersound
aickit conder
arese
aeedio
apprrest
aeartice spores
apprest
aeadio
araph
apprest
apprest
apprest
apprest
apprrest
arpserde


About half of them are the same as Argmax result. You can adjust temperature in one_hot function to adjust diversity.

<font color = 'darkblue'>'Aondersound' sounds like a new headphone or some new sound technology. 'Aeedio' sounds creative name. 'Araph' and 'Arese' are also good candidate for a new start-up name.</font>

By the way they all start with 'a'.

In [52]:
ct = 0
for name in SFnames:
    if name[0] == 'a':
        ct+=1
print(ct)
print(1/26)
print(ct/len(SFnames))

137
0.038461538461538464
0.058823529411764705


Slightly more companies starting with 'a' in the training set than it would be in a random selection 1/26.

In [53]:
from collections import defaultdict
ct = defaultdict(int)
for name in SFnames:
    ct[name[0]]+=1

In [60]:
df = pd.DataFrame.from_dict(ct, orient='index', columns = ['count'])

In [64]:
(df/len(SFnames)).sort_values(by='count', ascending = False)

,count
s,0.122799
c,0.083298
t,0.066982
p,0.065693
m,0.061829
b,0.060541
a,0.058824
l,0.045084
r,0.039502
g,0.039502


- Surprising that the model's first pick on the first letter is 'a' with zero initialization while 6 other characters appear more than 'a'.

### - Random sampling with non-zero initialization

Let's give some more variation using random initial states.

In [40]:
iteration = 20
for _ in range(iteration):
    x_initializer = np.zeros((1, 1, n_values))
    a_initializer = np.random.randn(1, n_a)*0.5
    c_initializer = np.random.randn(1, n_a)*0.5
#     a_initializer = np.zeros((1, n_a))
#     c_initializer = np.zeros((1, n_a))
    results, indices = predict_and_sample(output_model, x_initializer, a_initializer, c_initializer)
    for i in indices.flatten():
        if ix_to_char[i] == '\n':
            print()
            break
        print(ix_to_char[i],end="")
#     print("np.argmax(results[12]) =", np.argmax(results[12]))
#     print("np.argmax(results[17]) =", np.argmax(results[17]))
#     print("list(indices[12:18]) =", list(indices[12:18]))

ripprene
sporemant
eneestree
topers ate
hippriseal
@7sheappphayerable
adtate
appaaseare
heapprist
️igrom
bollabs
loond 
tretachen mectieng
rivens
trealist
crander
pppaster
harophate
hindersound
ted


<font color = 'darkblue'>We see 'Ted' which is an already existing name. ('Ted' is not in the training set.) 'Hindersound' definitely can be some product name or even a company name, 'Igrom' can fit to any field, and 'Rivens' can be a name of a game. Most of these names sound great for me! </font>